**Training a movie review classification model to distinguish between positive and negative reviews**

In [6]:
# Run this only when necessary
!git config --global user.name "nsahsimon"
!git config --global user.email "nsahsimonai01@gmail.com"
!git add .
!git commit -m "created movie review classifier"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/deep_learning_projects/RNN_projects/text_classification

/content/drive/MyDrive/deep_learning_projects/RNN_projects/text_classification


In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn
import os
import requests
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dropout, Dense
import tensorflow.keras.datasets as datasets


In [ ]:
data_url = http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz